# Train and deploy on Kubeflow from Notebooks

This notebook shows you how to use Kubeflow to build, train, and deploy models on Kubernetes.
This notebook walks you through the following
 
* Building an XGBoost model inside a notebook
* Training the model inside the notebook
* Performing inference using the model inside the notebook
* Using Kubeflow Fairing to launch training jobs on Kubernetes
* Using Kubeflow Fairing to build and deploy a model using [Seldon Core](https://www.seldon.io/)
* Using [Kubeflow metadata](https://github.com/kubeflow/metadata) to record metadata about your models
* Using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/) to build a pipeline to train your model

## Prerequisites 

* This notebook assumes you are running inside 0.6 Kubeflow deployed on GKE following the [GKE instructions](https://www.kubeflow.org/docs/gke/deploy/)
* If you are running somewhere other than GKE you will need to modify the notebook to use a different docker registry or else configure Kubeflow to work with GCR.

### Verify we have a GCP account

* The cell below checks that this notebook was spawned with credentials to access GCP
* To add credentials when you created the notebook you should have selected add gcp credential as shown below
  ![add credential](images/addgcpsecret.png)


In [1]:
import os
if not os.getenv("GOOGLE_APPLICATION_CREDENTIALS"):
    raise ValueError("Notebook is missing google application credentials")

## Install Required Libraries

Import the libraries required to train this model.

In [2]:
!pip3 install retrying

  Stored in directory: /home/jovyan/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


* Install a specific version of fairing that this example is tested against

In [3]:
!pip3 install git+git://github.com/kubeflow/fairing.git@c6c075dece72135f5883abfe2a296894d74a2367

  Cloning git://github.com/kubeflow/fairing.git (to revision c6c075dece72135f5883abfe2a296894d74a2367) to /tmp/pip-req-build-6xzul5lw
    100% |████████████████████████████████| 133kB 3.8MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 34.8MB/s ta 0:00:01
    100% |████████████████████████████████| 5.7MB 6.6MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 30.6MB/s ta 0:00:01


    100% |████████████████████████████████| 552kB 24.1MB/s ta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-jf08xe4p/wheels/bf/3f/0f/c65ae27dc4acd9443a98bfe546ad571e9a2d9f05905274ae44
Successfully built fairing
kfp 0.1 has requirement google-cloud-storage==1.13.0, but you'll have google-cloud-storage 1.14.0 which is incompatible.
kfp 0.1 has requirement kubernetes==8.0.0, but you'll have kubernetes 9.0.0 which is incompatible.
  Found existing installation: urllib3 1.24.1
    Uninstalling urllib3-1.24.1:
      Successfully uninstalled urllib3-1.24.1
  Found existing installation: fairing 0.5
    Uninstalling fairing-0.5:
      Successfully uninstalled fairing-0.5
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


* Perform some notebook setup

In [28]:
!gcloud

ERROR: (gcloud) Command name argument expected.
Command name argument expected.

Available commands for gcloud:

  AI and Machine Learning
      ml                      Use Google Cloud machine learning capabilities.
      ml-engine               Manage Cloud ML Engine jobs and models.

  API Platform and Ecosystems
      endpoints               Create, enable and manage API services.
      service-management      Create, enable and manage API services.
      services                List, enable and disable APIs and services.

  Compute
      app                     Manage your App Engine deployments.
      compute                 Create and manipulate Google Compute Engine
                              resources.
      container               Deploy and manage clusters of machines for running
                              containers.
      functions               Manage Google Cloud Functions.

  Data Analytics
      composer                Create and manage Cloud Composer Environment

In [4]:
import util
from pathlib import Path
import os

util.notebook_setup()

* Import the python libraries we will use
* We add a comment "fairing:include-cell" to tell the fairing preprocessor to keep this cell when converting to python code later

In [33]:
# fairing:include-cell
import fire
import joblib
import logging
import kfmd
import nbconvert
import os
import pathlib
import sys
from pathlib import Path
import pandas as pd
import pprint
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from importlib import reload
from sklearn.datasets import make_regression
from kfmd import metadata
from kfmd import openapi_client
from kfmd.openapi_client import Configuration, ApiClient, MetadataServiceApi
from datetime import datetime
import retrying
import urllib3


In [30]:
# Imports not to be included in the built docker image
import kfp
import kfp.components as comp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client
import fairing   
from fairing.builders import append
from fairing.deployers import job
from fairing.preprocessors.converted_notebook import ConvertNotebookPreprocessorWithFire
import yaml


## Code to train and predict 

* In the cells below we define some functions to generate data and train a model
* These functions could just as easily be defined in a separate python module

In [7]:
# fairing:include-cell
def read_synthetic_input(test_size=0.25):
    """generate synthetic data and split it into train and test."""
    # generate regression dataset
    X, y = make_regression(n_samples=200, n_features=5, noise=0.1)
    train_X, test_X, train_y, test_y = train_test_split(X,
                                                      y,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)


In [8]:
# fairing:include-cell
def train_model(train_X,
                train_y,
                test_X,
                test_y,
                n_estimators,
                learning_rate):
    """Train the model using XGBRegressor."""
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)

    model.fit(train_X,
            train_y,
            early_stopping_rounds=40,
            eval_set=[(test_X, test_y)])

    print("Best RMSE on eval: %.2f with %d rounds",
               model.best_score,
               model.best_iteration+1)
    return model

def eval_model(model, test_X, test_y):
    """Evaluate the model performance."""
    predictions = model.predict(test_X)
    mae=mean_absolute_error(predictions, test_y)
    logging.info("mean_absolute_error=%.2f", mae)
    return mae

def save_model(model, model_file):
    """Save XGBoost model for serving."""
    joblib.dump(model, model_file)
    logging.info("Model export success: %s", model_file)

@retrying.retry(stop_max_delay=180000)
def wait_for_istio(address="metadata-service.kubeflow.svc.cluster.local:8080"):
    """Wait until we can connect to the metadata service.
    
    When we launch a K8s pod we may not be able to connect to the metadata service immediately
    because the ISTIO side car hasn't started.
    
    This function allows us to wait for a time specified up to stop_max_delay to see if the service
    is ready.    
    """
    config = Configuration()
    config.host = address
    api_client = ApiClient(config)
    client = MetadataServiceApi(api_client)

    client.list_artifacts2()
    
def create_workspace():
    return metadata.Workspace(
        # Connect to metadata-service in namesapce kubeflow in k8s cluster.
        backend_url_prefix="metadata-service.kubeflow.svc.cluster.local:8080",
        name="xgboost-synthetic",
        description="workspace for xgboost-synthetic artifacts and executions")

## Wrap Training and Prediction in a class

* In the cell below we wrap training and prediction in a class
* A class provides the structure we will need to eventually use fairing to launch separate training jobs and/or deploy the model on Kubernetes

In [9]:
# fairing:include-cell
class ModelServe(object):    
    def __init__(self, model_file=None):
        self.n_estimators = 50
        self.learning_rate = 0.1
        if not model_file:
            if "MODEL_FILE" in os.environ:
                print("model_file not supplied; checking environment variable")
                model_file = os.getenv("MODEL_FILE")
            else:
                print("model_file not supplied; using the default")
                model_file = "mockup-model.dat"
        
        self.model_file = model_file
        print("model_file={0}".format(self.model_file))
        
        self.model = None
        self._workspace = None
        self.exec = self.create_execution()

    def train(self, **kwargs):
        print("Extra arguments: {0}".format(kwargs))
        (train_X, train_y), (test_X, test_y) = read_synthetic_input()
        
        # Here we use Kubeflow's metadata library to record information
        # about the training run to Kubeflow's metadata store.
        self.exec.log_input(metadata.DataSet(
            description="xgboost synthetic data",
            name="synthetic-data",
            owner="someone@kubeflow.org",
            uri="file://path/to/dataset",
            version="v1.0.0"))
        
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        mae = eval_model(model, test_X, test_y)
        
        # Here we log metrics about the model to Kubeflow's metadata store.
        self.exec.log_output(metadata.Metrics(
            name="xgboost-synthetic-traing-eval",
            owner="someone@kubeflow.org",
            description="training evaluation for xgboost synthetic",
            uri="gcs://path/to/metrics",
            metrics_type=metadata.Metrics.VALIDATION,
            values={"mean_absolute_error": mae}))
        
        save_model(model, self.model_file)
        self.exec.log_output(metadata.Model(
            name="housing-price-model",
            description="housing price prediction model using synthetic data",
            owner="someone@kubeflow.org",
            uri=self.model_file,
            model_type="linear_regression",
            training_framework={
                "name": "xgboost",
                "version": "0.9.0"
            },
            hyperparameters={
                "learning_rate": self.learning_rate,
                "n_estimators": self.n_estimators
            },
            version=datetime.utcnow().isoformat("T")))
        
    def predict(self, X, feature_names):
        """Predict using the model for given ndarray.
        
        The predict signature should match the syntax expected by Seldon Core
        https://github.com/SeldonIO/seldon-core so that we can use
        Seldon h to wrap it a model server and deploy it on Kubernetes
        """
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return [[prediction.item(0), prediction.item(1)]]

    @property
    def workspace(self):
        if not self._workspace:
            wait_for_istio()
            self._workspace = create_workspace()
        return self._workspace
    
    def create_execution(self):                
        r = metadata.Run(
            workspace=self.workspace,
            name="xgboost-synthetic-faring-run" + datetime.utcnow().isoformat("T"),
            description="a notebook run")

        return metadata.Execution(
            name = "execution" + datetime.utcnow().isoformat("T"),
            workspace=self.workspace,
            run=r,
            description="execution for training xgboost-synthetic")

## Train your Model Locally

* Train your model locally inside your notebook
* To train locally we just instatiante the ModelServe class and then call train

In [11]:
model = ModelServe(model_file="mockup-model.dat")
model.train()

model_file=mockup-model.dat
[0]	validation_0-rmse:100.138
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:93.6852
[2]	validation_0-rmse:89.0878
[3]	validation_0-rmse:86.0663
[4]	validation_0-rmse:82.7185
[5]	validation_0-rmse:79.9195
[6]	validation_0-rmse:76.812
[7]	validation_0-rmse:74.3731
[8]	validation_0-rmse:72.4495
[9]	validation_0-rmse:69.922
[10]	validation_0-rmse:68.6717
[11]	validation_0-rmse:67.5411
[12]	validation_0-rmse:66.2683
[13]	validation_0-rmse:64.8277
[14]	validation_0-rmse:63.7337
[15]	validation_0-rmse:62.0093
[16]	validation_0-rmse:60.1725
[17]	validation_0-rmse:58.6364
[18]	validation_0-rmse:57.159
[19]	validation_0-rmse:55.6644
[20]	validation_0-rmse:54.7235
[21]	validation_0-rmse:53.7659
[22]	validation_0-rmse:52.9136
[23]	validation_0-rmse:52.4023
[24]	validation_0-rmse:51.3452
[25]	validation_0-rmse:50.5797
[26]	validation_0-rmse:49.7565
[27]	validation_0-rmse:49.4316
[28]	validation_0-rmse:48.6402
[29]	validation_0-rms

mean_absolute_error=29.57
Model export success: mockup-model.dat


Best RMSE on eval: %.2f with %d rounds 40.118328 50


## Predict locally

* Run prediction inside the notebook using the newly created model
* To run prediction we just invoke redict

In [12]:
(train_X, train_y), (test_X, test_y) =read_synthetic_input()

ModelServe().predict(test_X, None)

model_file not supplied; using the default
model_file=mockup-model.dat


[[40.40393829345703, -183.3238525390625]]

## Use Fairing to Launch a K8s Job to train your model

* Now that we have trained a model locally we can use fairing to
  1. Launch a Kubernetes job to train the model
  1. Deploy the model on Kubernetes
* Launching a separate Kubernetes job to train the model has the following advantages

  * You can leverage Kubernetes to run multiple training jobs in parallel 
  * You can run long running jobs without blocking your kernel

### Configure The Docker Registry For Fairing

* In order to build docker images from your notebook we need a docker registry where the images will be stored
* Below you set some variables specifying a [GCR container registry](https://cloud.google.com/container-registry/docs/)
* Fairing provides a utility function to guess the name of your GCP project

In [97]:
# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(GCP_PROJECT)

## Use fairing to build the docker image

* First you will use fairing's kaniko builder to build a docker image that includes all your dependencies
  * You use kaniko because you want to be able to run `pip` to install dependencies
  * Kaniko gives you the flexibility to build images from Dockerfiles
* kaniko, however, can be slow
* so you will build a base image using Kaniko and then every time your code changes you will just build an image
  starting from your base image and adding your code to it
* you use the fairing build to enable these fast rebuilds

In [110]:
from fairing.builders import cluster
preprocessor = ConvertNotebookPreprocessorWithFire("ModelServe")

if not preprocessor.input_files:
    preprocessor.input_files = set()
input_files=["xgboost_util.py", "mockup-model.dat", "requirements.txt"]
preprocessor.input_files =  set([os.path.normpath(f) for f in input_files])
preprocessor.preprocess()

Converting build-train-deploy.ipynb to build-train-deploy.py
Creating entry point for the class name ModelServe


[PosixPath('build-train-deploy.py'),
 'requirements.txt',
 'xgboost_util.py',
 'mockup-model.dat']

### Build the base image

* You use cluster_builder to build the base image
* You only need to perform this again if we change our Docker image or the dependencies we need to install
* ClusterBuilder takes as input the DockerImage to use as a base image
* You should use the same Jupyter image that you are using for your notebook server so that your environment will be
  the same when you launch Kubernetes jobs

In [15]:
# Use a stock jupyter image as our base image
base_image = "gcr.io/kubeflow-images-public/tensorflow-1.13.1-notebook-cpu:v0.5.0"

cluster_builder = cluster.cluster.ClusterBuilder(registry=DOCKER_REGISTRY,
                                                 base_image=base_image,
                                                 preprocessor=preprocessor,
                                                 pod_spec_mutators=[fairing.cloud.gcp.add_gcp_credentials_if_exists],
                                                 context_source=cluster.gcs_context.GCSContextSource())
cluster_builder.build()

Building image using cluster builder.
Creating docker context: /tmp/fairing_context_p_cyutyy
Converting build-train-deploy.ipynb to build-train-deploy.py
Creating entry point for the class name ModelServe
Waiting for fairing-builder-gb2lp to start...
Waiting for fairing-builder-gb2lp to start...
Waiting for fairing-builder-gb2lp to start...
Waiting for fairing-builder-gb2lp to start...
Waiting for fairing-builder-gb2lp to start...
Pod started running True


INFO[0006] Downloading base image gcr.io/kubeflow-images-public/tensorflow-1.13.1-notebook-cpu:v0.5.0
INFO[0006] Downloading base image gcr.io/kubeflow-images-public/tensorflow-1.13.1-notebook-cpu:v0.5.0
WARN[0006] Error while retrieving image from cache: getting image from path: open /cache/sha256:5aaccf0267f085afd976342a8e943a9c6cefccef5b554df4e15fa7bf15cbd7a3: no such file or directory
INFO[0006] Using files from context: [/kaniko/buildcontext/app/requirements.txt]
INFO[0006] Checking for cached layer gcr.io/jlewi-dev/fairing-job/fairing-job/cache:488afeebb3bcde81c38dc99f3de2f4ba06b85302f8440b625e8fe1b08c53ef72...
INFO[0007] Using caching version of cmd: RUN if [ -e requirements.txt ];then pip install --no-cache -r requirements.txt; fi
INFO[0007] Using files from context: [/kaniko/buildcontext/app]
INFO[0007] Taking snapshot of full filesystem...
INFO[0007] Skipping paths under /dev, as it is a whitelisted directory
INFO[0007] Skipping paths under /etc/secrets, as it is a whiteliste

### Build the actual image

Here you use the append builder to add your code to the base image

* Calling preprocessor.preprocess() converts your notebook file to a python file

  * You are using the [ConvertNotebookPreprocessorWithFire](https://github.com/kubeflow/fairing/blob/master/fairing/preprocessors/converted_notebook.py#L85) 
  * This preprocessor converts ipynb files to py files by doing the following
    1. Removing all cells which don't have a comment `# fairing:include-cell`
    1. Using [python-fire](https://github.com/google/python-fire) to add entry points for the class specified in the constructor 
    
  * Call preprocess() will create the file build-train-deploy.py
  
* You use the AppendBuilder to rapidly build a new docker image by quickly adding some files to an existing docker image
  * The AppendBuilder is super fast so its very convenient for rebuilding your images as you iterate on your code
  * The AppendBuilder will add the converted notebook, build-train-deploy.py, along with any files specified in `preprocessor.input_files` to `/app` in the newly created image

In [111]:
preprocessor.preprocess()

builder = append.append.AppendBuilder(registry=DOCKER_REGISTRY,
                                      base_image=cluster_builder.image_tag, preprocessor=preprocessor)
builder.build()


Converting build-train-deploy.ipynb to build-train-deploy.py
Creating entry point for the class name ModelServe
Building image using Append builder...
Creating docker context: /tmp/fairing_context_ofq601li
Converting build-train-deploy.ipynb to build-train-deploy.py
Creating entry point for the class name ModelServe
Loading Docker credentials for repository 'gcr.io/jlewi-dev/fairing-job/fairing-job:8FB954EE'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Image successfully built in 1.1783705039415509s.
Pushing image gcr.io/jlewi-dev/fairing-job/fairing-job:A2EFFE46...
Loading Docker credentials for repository 'gcr.io/jlewi-dev/fairing-job/fairing-job:A2EFFE46'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Uploading gcr.io/jlewi-dev/fairing-job/fairing-job:A2EFFE46
Layer sha256:147c5bbff888fc9cddffd4078daa35bba0d1d6f6c7175a1acb144412a43b3fce exists, skipping
Layer s

## Launch the K8s Job

* You can use fairing to easily launch a [Kubernetes job](https://kubernetes.io/docs/concepts/workloads/controllers/jobs-run-to-completion/) to invoke code
* You use fairings Kubernetes job library to build a Kubernetes job
  * You use pod mutators to attach GCP credentials to the pod
  * You can also use pod mutators to attch PVCs
* Since the [ConvertNotebookPreprocessorWithFire](https://github.com/kubeflow/fairing/blob/master/fairing/preprocessors/converted_notebook.py#L85) is using [python-fire](https://github.com/google/python-fire) you can easily invoke any method inside the ModelServe class just by configuring the command invoked by the Kubernetes job
   * In the cell below you extend the command to include `train` as an argument because you want to invoke the train
     function
     
**Note** When you invoke train_deployer.deploy; fairing will stream the logs from the Kubernetes job. The job will initially show some connection errors because the job will try to connect to the metadataserver. You can ignore these errors; the job will retry until its able to connect and then continue

In [17]:
pod_spec = builder.generate_pod_spec()
train_deployer = job.job.Job(cleanup=False,
                             pod_spec_mutators=[
                             fairing.cloud.gcp.add_gcp_credentials_if_exists])

# Add command line arguments
pod_spec.containers[0].command.extend(["train"])
result = train_deployer.deploy(pod_spec)

The job fairing-job-x8sbq launched.
Waiting for fairing-job-x8sbq-wgjzc to start...
Waiting for fairing-job-x8sbq-wgjzc to start...
Waiting for fairing-job-x8sbq-wgjzc to start...
Waiting for fairing-job-x8sbq-wgjzc to start...
Waiting for fairing-job-x8sbq-wgjzc to start...
Pod started running True


Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c8e7a24a8>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c8e7a2588>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c8e7a2668>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.con

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f3f4630>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f3f4550>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f41a828>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.con

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f44ab70>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f44ac50>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f44ad30>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.con

Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f45be80>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f45bd68>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f1ac9b0>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.con

Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f45bdd8>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f45bd68>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f45b780>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.con

Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c8e7a2dd8>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f429a58>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c7f429be0>: Failed to establish a new connection: [Errno 111] Connection refused',)': /api/v1alpha1/artifacts
Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.con

* You can use kubectl to inspect the job that fairing created

In [18]:
!kubectl get jobs -l fairing-id={train_deployer.job_id} -o yaml

apiVersion: v1
items:
- apiVersion: batch/v1
  kind: Job
  metadata:
    creationTimestamp: "2019-09-12T04:38:29Z"
    generateName: fairing-job-
    labels:
      fairing-deployer: job
      fairing-id: 2b17f22a-d517-11e9-b95f-f6ae68c06c0f
    name: fairing-job-x8sbq
    namespace: jlewi-kf
    resourceVersion: "1042157"
    selfLink: /apis/batch/v1/namespaces/jlewi-kf/jobs/fairing-job-x8sbq
    uid: 2b1ca4d7-d517-11e9-a836-42010a8e00aa
  spec:
    backoffLimit: 0
    completions: 1
    parallelism: 1
    selector:
      matchLabels:
        controller-uid: 2b1ca4d7-d517-11e9-a836-42010a8e00aa
    template:
      metadata:
        creationTimestamp: null
        labels:
          controller-uid: 2b1ca4d7-d517-11e9-a836-42010a8e00aa
          fairing-deployer: job
          fairing-id: 2b17f22a-d517-11e9-b95f-f6ae68c06c0f
          job-name: fairing-job-x8sbq
        name: fairing-deployer
      spec:
        containers:
        - command:
          - python
          - /app/build-trai

## Do hyperparameter tuning

* Run hyperparameter tuning on the model.

* Define the hypterparameter tuning Job
* Spec for [Experiment](https://github.com/kubeflow/katib/blob/d57109455ec6e64d64d81d210d64b97156b468d9/pkg/apis/controller/experiments/v1alpha2/experiment_types.go#L24)

In [99]:
builder.image_tag

'gcr.io/jlewi-dev/fairing-job/fairing-job:82C84ACE'

In [101]:
preprocessor.executable.name

'build-train-deploy.py'

In [113]:
hp_experiment_raw = """
apiVersion: "kubeflow.org/v1alpha2"
kind: Experiment
metadata:  
  labels:
    example: xgboost-demo
spec:
  parallelTrialCount: 3
  maxTrialCount: 3
  maxFailedTrialCount: 3
  objective:
    type: maximize
    goal: 0.99
    objectiveMetricName: Validation-accuracy
    additionalMetricNames:
      - accuracy
  algorithm:
    algorithmName: random
  trialTemplate:
    goTemplate:
      rawTemplate: ""
  parameters:
    - name: --num-layers
      parameterType: int
      feasibleSpace:
        min: "1"
        max: "4"
"""        

# The batch job that will be launched on each trial
# 
trial_job_raw = """apiVersion: batch/v1
kind: Job
metadata:
  name: {{.Trial}}
  namespace: {{.NameSpace}}
  labels:
    demo: xgboost
    hpjob: xgboost
spec:
  template:
    metadata:
      annotations:
        sidecar.istio.io/inject: "false"
    spec:
      containers:
      - name: {{.Trial}}
        image: gcr.io/jlewi-dev/fairing-job/fairing-job:A2EFFE46
        workingDir: /app
        command:
        - "python"
        - "build-train-deploy.py"
        - train
        - "--batch-size=64"
        {{- with .HyperParameters}}
        {{- range .}}
        - "{{.Name}}={{.Value}}"
        {{- end}}
        {{- end}}
      restartPolicy: Never
"""

# format messes up escaping because the curly braces get replaced.
# trial_job = trial_job_raw.format(image=builder.image_tag)
hp_experiment = yaml.load(hp_experiment_raw)
hp_experiment["metadata"]["namespace"] = fairing.utils.get_current_k8s_namespace()

hp_experiment["spec"]["trialTemplate"]["goTemplate"]["rawTemplate"] = trial_job_raw

# Set the name for the job

import datetime
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
hp_experiment["metadata"]["name"] = "xgboost-{0}".format(now)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:64: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [114]:
client = k8s_client.ApiClient()
crd_api = k8s_client.CustomObjectsApi(client)

group, version = hp_experiment['apiVersion'].split('/')
#print(yaml.safe_dump(hp_experiment))


result = crd_api.create_namespaced_custom_object(
  group=group,
  version=version,
  namespace=hp_experiment["metadata"]["namespace"],
  plural='experiments',
  body=hp_experiment)


In [65]:
result = crd_api.get_namespaced_custom_object(
  group=group,
  version=version,
  namespace=hp_experiment["metadata"]["namespace"],
  plural='experiments',
  name="random-experiment")


In [88]:
print(yaml.safe_dump(hp_experiment["spec"]["trialTemplate"]))

goTemplate:
  rawTemplate: "apiVersion: batch/v1\nkind: Job\nmetadata:\n  name: {.Trial}\n  namespace:\
    \ {.NameSpace}\nspec:\n  template:\n    spec:\n      containers:\n      - name:\
    \ {.Trial}\n        image: gcr.io/jlewi-dev/fairing-job/fairing-job:503200D3\n\
    \        command:\n        - \"python\"\n        - \"/mxnet/example/image-classification/train_mnist.py\"\
    \n        - \"--batch-size=64\"\n        {- with .HyperParameters}\n        {-\
    \ range .}\n        - \"{.Name}={.Value}\"\n        {- end}\n        {- end}\n\
    \      restartPolicy: Never\n"



In [79]:
print(yaml.safe_dump(result["spec"]["trialTemplate"]))

goTemplate:
  rawTemplate: "apiVersion: batch/v1\nkind: Job\nmetadata:\n  name: {{.Trial}}\n \
    \ namespace: {{.NameSpace}}\nspec:\n  template:\n    spec:\n      containers:\n\
    \      - name: {{.Trial}}\n        image: katib/mxnet-mnist-example\n        command:\n\
    \        - \"python\"\n        - \"/mxnet/example/image-classification/train_mnist.py\"\
    \n        - \"--batch-size=64\"\n        {{- with .HyperParameters}}\n       \
    \ {{- range .}}\n        - \"{{.Name}}={{.Value}}\"\n        {{- end}}\n     \
    \   {{- end}}\n      restartPolicy: Never"



## Deploy the trained model to Kubeflow for predictions

* Now that you have trained a model you can use fairing to deploy it on Kubernetes
* When you call deployer.deploy fairing will create a [Kubernetes Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/) to serve your model
* Fairing uses the docker image you created earlier
* The docker image you created contains your code and [Seldon core](https://www.seldon.io/)
* Fairing uses Seldon to wrap your prediction code, ModelServe.predict, in a REST and gRPC server

In [19]:
from fairing.deployers import serving
pod_spec = builder.generate_pod_spec()

module_name = os.path.splitext(preprocessor.executable.name)[0]
deployer = serving.serving.Serving(module_name + ".ModelServe",
                                   service_type="ClusterIP",
                                   labels={"app": "mockup"})
    
url = deployer.deploy(pod_spec)

Cluster endpoint: http://fairing-service-2522n.jlewi-kf.svc.cluster.local


* You can use kubectl to inspect the deployment that fairing created

In [20]:
!kubectl get deploy -o yaml {deployer.deployment.metadata.name}

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
  creationTimestamp: "2019-09-12T04:41:07Z"
  generateName: fairing-deployer-
  generation: 1
  labels:
    app: mockup
    fairing-deployer: serving
    fairing-id: 89159f80-d517-11e9-b95f-f6ae68c06c0f
  name: fairing-deployer-94ns7
  namespace: jlewi-kf
  resourceVersion: "1043015"
  selfLink: /apis/extensions/v1beta1/namespaces/jlewi-kf/deployments/fairing-deployer-94ns7
  uid: 891925d1-d517-11e9-a836-42010a8e00aa
spec:
  progressDeadlineSeconds: 600
  replicas: 1
  revisionHistoryLimit: 10
  selector:
    matchLabels:
      app: mockup
      fairing-deployer: serving
      fairing-id: 89159f80-d517-11e9-b95f-f6ae68c06c0f
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: mockup
        fairing-deployer: serving
        fairing-id: 89159f

## Send an inference request to the prediction server

* Now that you have deployed the model into your Kubernetes cluster, you can send a REST request to 
  preform inference
* The code below reads some data, sends, a prediction request and then prints out the response

In [21]:
(train_X, train_y), (test_X, test_y) = read_synthetic_input()


In [22]:
full_url = url + ":5000/predict"
result = util.predict_nparray(full_url, test_X)
pprint.pprint(result.content)

(b'{"data":{"names":["t:0","t:1"],"tensor":{"shape":[1,2],"values":[-18.9192104'
 b'3395996,73.20831298828125]}},"meta":{}}\n')


## Clean up the prediction endpoint

* You can use kubectl to delete the Kubernetes resources for your model
* If you want to delete the resources uncomment the following lines and run them

In [23]:
# !kubectl delete service -l app=ames
# !kubectl delete deploy -l app=ames

## Track Models and Artifacts

* Using Kubeflow's metadata server you can track models and artifacts
* The ModelServe code was instrumented to log executions and outputs
* You can access Kubeflow's metadata UI by selecting **Artifact Store** from the central dashboard
  * See [here](https://www.kubeflow.org/docs/other-guides/accessing-uis/) for instructions on connecting to Kubeflow's UIs
* You can also use the python SDK to read and write entries
* This [notebook](https://github.com/kubeflow/metadata/blob/master/sdk/python/demo.ipynb) illustrates a bunch of metadata functionality

### Create a workspace

* Kubeflow metadata uses workspaces as a logical grouping for artifacts, executions, and datasets that belong together
* Earlier in the notebook we defined the function `create_workspace` to create a workspace for this example
* You can use that function to return a workspace object and then call list to see all the artifacts in that workspace

In [24]:
ws = create_workspace()
ws.list()

[{'id': '3',
  'workspace': 'xgboost-synthetic',
  'run': 'xgboost-synthetic-faring-run2019-09-12T04:37:44.053070',
  'create_time': '2019-09-12T04:37:44.217218Z',
  'description': 'housing price prediction model using synthetic data',
  'model_type': 'linear_regression',
  'name': 'housing-price-model',
  'owner': 'someone@kubeflow.org',
  'version': '2019-09-12T04:37:44.217202',
  'uri': 'mockup-model.dat',
  'training_framework': {'name': 'xgboost', 'version': '0.9.0'},
  'hyperparameters': {'learning_rate': 0.1, 'n_estimators': 50},
  'labels': None},
 {'id': '6',
  'workspace': 'xgboost-synthetic',
  'run': 'xgboost-synthetic-faring-run2019-09-12T04:41:01.121794',
  'create_time': '2019-09-12T04:41:01.256718Z',
  'description': 'housing price prediction model using synthetic data',
  'model_type': 'linear_regression',
  'name': 'housing-price-model',
  'owner': 'someone@kubeflow.org',
  'version': '2019-09-12T04:41:01.256698',
  'uri': 'mockup-model.dat',
  'training_framework': {

## Create a pipeline to train your model

* [Kubeflow pipelines](https://www.kubeflow.org/docs/pipelines/) makes it easy to define complex workflows to build and deploy models
* Below you will define and run a simple one step pipeline to train your model
* Kubeflow pipelines uses experiments to group different runs of a pipeline together
* So you start by defining a name for your experiement

#### Define the pipeline

* To create a pipeline you create a function and decorate it with the `@dsl.pipeline` decorator
  * You use the decorator to give the pipeline a name and description
  
* Inside the function, each step in the function is defined by a ContainerOp that specifies
  a container to invoke
 
* You will use the container image that you built earlier using Fairing
* Since the Fairing preprocessor added a main function using [python-fire](https://github.com/google/python-fire), a step in your pipeline can invocation any function in the ModelServe class just by setting the command for the container op
* See the pipelines [SDK reference](https://kubeflow-pipelines.readthedocs.io/en/latest/) for more information

In [25]:
@dsl.pipeline(
   name='Training pipeline',
   description='A pipeline that trains an xgboost model for the Ames dataset.'
)
def train_pipeline(
   ):      
    command=["python", preprocessor.executable.name, "train"]
    train_op = dsl.ContainerOp(
            name="train", 
            image=builder.image_tag,        
            command=command,
            ).apply(
                gcp.use_gcp_secret('user-gcp-sa'),
            )
    train_op.container.working_dir = "/app"

#### Compile the pipeline

* Pipelines need to be compiled

In [26]:
pipeline_func = train_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

* Pipelines groups runs using experiments
* So before you submit a pipeline you need to create an experiment or pick an existing experiment
* Once you have compiled a pipeline, you can use the pipelines SDK to submit that pipeline


In [27]:
EXPERIMENT_NAME = 'MockupModel'

#Specify pipeline argument values
arguments = {}

# Get or create an experiment and submit a pipeline run
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#vvvvvvvvv This link leads to the run information page. (Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working)

Creating experiment MockupModel.
